In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import nltk
import pandas as pd
import os
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [51]:
##StopWords Preprocessing
inputs = []
for file in os.listdir("/content/drive/MyDrive/sentiment Analysis/StopWords"):
    if file.endswith(".txt"):
        inputs.append(os.path.join("/content/drive/MyDrive/sentiment Analysis/StopWords", file))


# concatanate all txt files in a file called merged_file.txt
with open('/content/drive/MyDrive/sentiment Analysis/StopWords/StopWordMerged_file.txt', 'w') as outfile:
    for fname in inputs:
        with open(fname, encoding="utf-8", errors='ignore') as infile:
            outfile.write(infile.read())



sw = open('/content/drive/MyDrive/sentiment Analysis/StopWords/StopWordMerged_file.txt', 'r')
stopword_data=sw.read()
stopword_text=stopword_data.split('\n')
stopword_text

clean_text_stopwords=[]
def to_lower_case_stopwords(data):
  for words in stopword_text:
    clean_text_stopwords.append(str.lower(words))
to_lower_case_stopwords(stopword_text)


In [52]:
##Positive words and Negative words Preprocessing
pw = open('/content/drive/MyDrive/sentiment Analysis/MasterDictionary/positive-words.txt', 'r')
positive_word_data=pw.read()
positive_words=positive_word_data.split('\n')

nw = open('/content/drive/MyDrive/sentiment Analysis/MasterDictionary/negative-words.txt', 'r',encoding="utf-8")
negative_word_data=nw.read()
negative_words=negative_word_data.split('\n')

In [53]:
##Text Analysis function
def text_analysis(file):
  f = open(file, 'r')
  data=f.read()
  raw_text=data.split('\n')

  ##lower case text conversion
  clean_text_1=[]
  def to_lower_case(data):
    for words in raw_text:
      clean_text_1.append(str.lower(words))
  to_lower_case(raw_text)

  #Tokenization of words in text
  clean_text_2=[word_tokenize(i) for i in clean_text_1]

  #Cleaning of punctuations in text
  clean_text_3=[]
  for words in clean_text_2:
    clean=[]
    for w in words:
      res=re.sub(r'[^\w\s]',"",w)
      if res != "":
        clean.append(res)
    clean_text_3.append(clean)

  #Cleaning of stopwords in text
  clean_text_4=[]
  for  words in clean_text_3:
    w=[]
    for word in words:
      if not word in clean_text_stopwords:

        w.append(word)
    clean_text_4.append(w)

  preprocessed_data=clean_text_4

  #Calculation of positive_score,negative_score,word_count,polarity_score&subjectivity_score
  positive_score=0
  negative_score=0
  neutral_score=0

  for words in preprocessed_data:
    for word in words:
      if word in positive_words:
        positive_score+=1
      elif word in negative_words:
        negative_score+=1
      else:
        neutral_score+=1

  total_word_count_cleaned=positive_score+negative_score+neutral_score

  polarity_score=(positive_score-negative_score)/((positive_score+negative_score)+0.000001)

  subjectivity_score=(positive_score+negative_score)/((total_word_count_cleaned)+0.000001)



  num_of_sentences=len(clean_text_3)

  word_count=0
  for words in clean_text_3:
    for word in words:
      word_count+=1

  #Calculation of Average Sentence Length
  avg_sen_len=word_count/num_of_sentences

  syllable_count=[]
  for words in clean_text_3:
    for word in words:
      count=0
      for char in word:
        if char in 'aeiou':
          count+=1
      if word[-2:]=='ed' or word[-2:]=='es':
        count-=1
      syllable_count.append(count)




  # Calculation of syllable_per_word, complex_word_count, percentage_of_complex_words & fog index
  syllable_per_word=sum(syllable_count)/len(syllable_count)


  complex_words_syllable=[i for i in syllable_count if i>2]
  complex_word_count=len(complex_words_syllable)
  percentage_of_complex_words=complex_word_count/word_count

  fog_index=0.4*(avg_sen_len+percentage_of_complex_words)

  ##Calculation of Average Word length
  char_count=0
  for words in clean_text_3:
    for word in words:
      char_count+=len(word)

  avg_word_len=char_count/word_count


  #Calculation of personel pronoun count
  personel_pronoun_text=[word_tokenize(i) for i in raw_text]
  # import re
  personel_pronoun_text_3=[]

  for words in personel_pronoun_text:
    clean=[]
    for w in words:
      res=re.sub(r'[^\w\s]',"",w)
      if res != "":
        clean.append(res)
    personel_pronoun_text_3.append(clean)


  personel_pronoun_text_3_updated=[]
  for words in personel_pronoun_text_3:
    for word in words:
      personel_pronoun_text_3_updated.append(word)

  personel_pronouns=["I","we","my","ours","us"]
  personel_pronouns_count=0
  for i in personel_pronouns:
    personel_pronouns_count+=personel_pronoun_text_3_updated.count(i)


  # print("1.Positive Score:",positive_score)
  # print("2.Negative_Score:",negative_score)
  # print("3.Polarity_Score:",polarity_score)
  # print("4.Subjectivity_Score:",subjectivity_score)
  # print("5.Average Sentence Length:",avg_sen_len)
  # print("6.Percentage of complex words:",percentage_of_complex_words*100)
  # print("7.Fog Index:",fog_index)
  # print("8.Average number of words per sentence:",avg_sen_len)
  # print("9.Complex Word Count",complex_word_count)
  # print("10.Word Count:",total_word_count_cleaned)
  # print("11.Syllable per word:",syllable_per_word)
  # print("12.Personel Pronouns:",personel_pronouns_count)
  # print("13.Average Word Length:",avg_word_len)

  return positive_score,negative_score,polarity_score,subjectivity_score,avg_sen_len,percentage_of_complex_words*100,fog_index,avg_sen_len,complex_word_count,total_word_count_cleaned,syllable_per_word,personel_pronouns_count,avg_word_len

In [54]:

# def text_analysis(file):
#   f = open(file, 'r')
#   data=f.read()
#   raw_text=data.split('\n')


#   clean_text_1=[]
#   def to_lower_case(data):
#     for words in raw_text:
#       clean_text_1.append(str.lower(words))
#   to_lower_case(raw_text)

#   clean_text_2=[word_tokenize(i) for i in clean_text_1]

#   clean_text_3=[]

#   for words in clean_text_2:
#     clean=[]
#     for w in words:
#       res=re.sub(r'[^\w\s]',"",w)
#       if res != "":
#         clean.append(res)
#     clean_text_3.append(clean)


#   clean_text_4=[]
#   for  words in clean_text_3:
#     w=[]
#     for word in words:
#       if not word in clean_text_stopwords:

#         w.append(word)
#     clean_text_4.append(w)

#   preprocessed_data=clean_text_4

#   positive_score=0
#   negative_score=0
#   neutral_score=0

#   for words in preprocessed_data:
#     for word in words:
#       if word in positive_words:
#         positive_score+=1
#       elif word in negative_words:
#         negative_score+=1
#       else:
#         neutral_score+=1

#   total_word_count_cleaned=positive_score+negative_score+neutral_score

#   polarity_score=(positive_score-negative_score)/((positive_score+negative_score)+0.000001)

#   subjectivity_score=(positive_score+negative_score)/((total_word_count_cleaned)+0.000001)



#   num_of_sentences=len(clean_text_3)

#   word_count=0
#   for words in clean_text_3:
#     for word in words:
#       word_count+=1


#   avg_sen_len=word_count/num_of_sentences

#   syllable_count=[]
#   for words in clean_text_3:
#     for word in words:
#       count=0
#       for char in word:
#         if char in 'aeiou':
#           count+=1
#       if word[-2:]=='ed' or word[-2:]=='es':
#         count-=1
#       syllable_count.append(count)




#   # print("sum",sum(syllable_count))
#   syllable_per_word=sum(syllable_count)/len(syllable_count)


#   complex_words_syllable=[i for i in syllable_count if i>2]
#   complex_word_count=len(complex_words_syllable)
#   percentage_of_complex_words=complex_word_count/word_count

#   fog_index=0.4*(avg_sen_len+percentage_of_complex_words)

#   ##Average Word length
#   char_count=0
#   for words in clean_text_3:
#     for word in words:
#       char_count+=len(word)

#   avg_word_len=char_count/word_count



#   personel_pronoun_text=[word_tokenize(i) for i in raw_text]
#   # import re
#   personel_pronoun_text_3=[]

#   for words in personel_pronoun_text:
#     clean=[]
#     for w in words:
#       res=re.sub(r'[^\w\s]',"",w)
#       if res != "":
#         clean.append(res)
#     personel_pronoun_text_3.append(clean)


#   personel_pronoun_text_3_updated=[]
#   for words in personel_pronoun_text_3:
#     for word in words:
#       personel_pronoun_text_3_updated.append(word)

#   personel_pronouns=["I","we","my","ours","us"]
#   personel_pronouns_count=0
#   for i in personel_pronouns:
#     personel_pronouns_count+=personel_pronoun_text_3_updated.count(i)


#   # print("1.Positive Score:",positive_score)
#   # print("2.Negative_Score:",negative_score)
#   # print("3.Polarity_Score:",polarity_score)
#   # print("4.Subjectivity_Score:",subjectivity_score)
#   # print("5.Average Sentence Length:",avg_sen_len)
#   # print("6.Percentage of complex words:",percentage_of_complex_words*100)
#   # print("7.Fog Index:",fog_index)
#   # print("8.Average number of words per sentence:",avg_sen_len)
#   # print("9.Complex Word Count",complex_word_count)
#   # print("10.Word Count:",total_word_count_cleaned)
#   # print("11.Syllable per word:",syllable_per_word)
#   # print("12.Personel Pronouns:",personel_pronouns_count)
#   # print("13.Average Word Length:",avg_word_len)


#   return positive_score,negative_score,polarity_score,subjectivity_score,avg_sen_len,percentage_of_complex_words*100,fog_index,avg_sen_len,complex_word_count,total_word_count_cleaned,syllable_per_word,personel_pronouns_count,avg_word_len

In [55]:
#Text files extraction and final output updata in Output data file
output_file=pd.read_excel("/content/drive/MyDrive/sentiment Analysis/Output Data Structure.xlsx")
for i in output_file["URL_ID"]:
  file='/content/drive/MyDrive/sentiment Analysis/input_text_data/'+str(i)+'.txt'
  positive_score,negative_score,polarity_score,subjectivity_score,avg_sen_len,percentage_of_complex_words,fog_index,avg_sen_len,complex_word_count,total_word_count_cleaned,syllable_per_word,personel_pronouns_count,avg_word_len=text_analysis(file)
  output_file.loc[output_file.URL_ID==i,["POSITIVE SCORE","NEGATIVE SCORE","POLARITY SCORE","SUBJECTIVITY SCORE","AVG SENTENCE LENGTH","PERCENTAGE OF COMPLEX WORDS","FOG INDEX","AVG NUMBER OF WORDS PER SENTENCE","COMPLEX WORD COUNT","WORD COUNT","SYLLABLE PER WORD","PERSONAL PRONOUNS","AVG WORD LENGTH"]]=[positive_score,negative_score,polarity_score,subjectivity_score,avg_sen_len,percentage_of_complex_words,fog_index,avg_sen_len,complex_word_count,total_word_count_cleaned,syllable_per_word,personel_pronouns_count,avg_word_len]
  print(i)

output_file.to_excel("Output Data Structure Updated.xlsx")

37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
